In [13]:
import ee
import geemap
import datetime

# Initialize Earth Engine
ee.Initialize()

# Define Nachole Upazila as a rectangle or use a shapefile
nachole = ee.FeatureCollection("projects/gee-myproject-israqsadmani/assets/Nachole")

# Define date range
start_date = '2025-01-15'
end_date = '2025-02-15'

# Load Sentinel-2 ImageCollection
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(nachole) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE')

# Print image info
#print('Number of images:', s2.size().getInfo())

# Get the least cloudy image
image = s2.first()


# Visualize in RGB
Map = geemap.Map()
Map.centerObject(nachole, 12)
Map.addLayer(image.clip(nachole), {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}, 'Sentinel-2 RGB')
Map.addLayer(nachole, {}, 'Nachole Boundary')
Map


Map(center=[24.722085103316534, 88.39499976812247], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
# Get the least cloudy image
first_image = s2.first()

# Clip to Nachole AOI
clipped_image = first_image.clip(nachole)

In [15]:
export_task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='Nachole_S2_FullBands',
    folder='GEE_Exports',
    fileNamePrefix='nachole_s2_jan_feb_2025_all_bands',
    region=nachole.geometry(),
    scale=10,  # Sentinel-2 native resolution
    crs='EPSG:4326',
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# Start the export
export_task.start()
